In [1]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm


# CS Rankings Author info

Taken from https://github.com/emeryberger/CSrankings/tree/gh-pages which is the repo behind https://csrankings.org 

Run `make generated-author-info.csv` which should make the csv. Each row contains the score for every author, year, and conference combination. This means that there are multiple rows for each professor.

# CS Professors

Taken from https://drafty.cs.brown.edu/csprofessors but they hide the data. So, after reading source code, exploit a [thing they left in](https://github.com/brownhci/drafty/blob/212bd995c857a34c74c7a71d67e1556c1ca7ea97/backend/src/controllers/datasharing.ts#L31) during development and use https://drafty.cs.brown.edu/data/csv/csprofessors/csprofessors_93318b344889ccef41d46b5f83d63de5 

# Placement Rank

Taken from https://drafty.cs.brown.edu/csopenrankings/placement-rank.html which is just copy and paste (and `M-x query-replace <tab> ,`). I think I could have done this myself, but why do that when someone else has already done the work. 

# Best Paper awards

A collection of best paper awards are listed on https://jeffhuang.com/best_paper_awards/

However, I needed to do some html parsing in order to get the data into a csv format. That is done in [another notebook](./best-paper.ipynb)

# Preferences
This is a list of conferences and areas and my interest in them. This is used in order to create weights. It is manually generated

# Universities
This is a list of north american universities and their aliases (e.g. NYU and New York University). It is manually generated (hence why it is only north american universities)

# [Data](https://store.sachiniyer.com/s/9NczHAmPJFiQgys)


In [2]:
dfcsrankings = pd.read_csv("data/generated-author-info.csv")
dfcsrankings.describe()

,count,adjustedcount,year
count,199906.000000,199906.000000,199906.000000
mean,1.332511,0.384866,2012.689984
std,0.845756,0.281289,8.774593
min,1.000000,0.008696,1970.000000
25%,1.000000,0.200000,2008.000000
50%,1.000000,0.333330,2015.000000
75%,1.000000,0.500000,2020.000000
max,24.000000,4.977800,2023.000000


In [3]:
dfcsrankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199906 entries, 0 to 199905
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           199906 non-null  object 
 1   dept           199906 non-null  object 
 2   area           199906 non-null  object 
 3   count          199906 non-null  float64
 4   adjustedcount  199906 non-null  float64
 5   year           199906 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 9.2+ MB


In [4]:
dfcsrankings.sample(3)

,name,dept,area,count,adjustedcount,year
56026,Gilbert L. Peterson,Air Force Institute of Technology,iros,1.0,0.50,2011
151777,Seth Gilbert,National University of Singapore,ec,1.0,0.25,2006
50021,Fei Chao 0001,Xiamen University,cvpr,1.0,0.10,2021


In [5]:
dfcsprofs = pd.read_csv("data/csprofessors.csv")
dfcsprofs.describe()

,UniqueId,JoinYear
count,5630.000000,5512.000000
mean,3098.262877,2007.741473
std,1784.465144,11.730682
min,1.000000,1963.000000
25%,1569.250000,2001.000000
50%,3081.500000,2011.000000
75%,4623.750000,2017.000000
max,6212.000000,2024.000000


In [6]:
dfcsprofs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   UniqueId    5630 non-null   int64  
 1   FullName    5629 non-null   object 
 2   University  5630 non-null   object 
 3   JoinYear    5512 non-null   float64
 4   SubField    5625 non-null   object 
 5   Bachelors   5331 non-null   object 
 6   Doctorate   5618 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 308.0+ KB


In [7]:
dfcsprofs.sample(3)

,UniqueId,FullName,University,JoinYear,SubField,Bachelors,Doctorate
2824,3091,Ben Y. Zhao,University of Chicago,2017.0,Machine Learning & Data Mining,Yale University,"University of California, Berkeley"
1712,1921,W. Randolph Franklin,Rensselaer Polytechnic Institute,1978.0,Computer Graphics,University of Toronto,Harvard University
3506,3841,Stephen McCamant,University of Minnesota,2012.0,Computer Security,"University of California, Berkeley",Massachusetts Institute of Technology


In [8]:
dfplacement = pd.read_csv("data/placement-rank.csv", delimiter='|')
dfplacement.describe()

,num,size,bachelors,doctorate,placementscore
count,183.000000,115.000000,168.000000,159.000000,183.000000
mean,92.000000,45.121739,84.500000,80.000000,1.523333
std,52.971691,33.426050,48.641546,46.043458,1.664107
min,1.000000,1.000000,1.000000,1.000000,0.820000
25%,46.500000,22.500000,42.750000,40.500000,0.865000
50%,92.000000,39.000000,84.500000,80.000000,1.000000
75%,137.500000,60.000000,126.250000,119.500000,1.415000
max,183.000000,253.000000,168.000000,159.000000,15.400000


In [9]:
dfplacement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   num             183 non-null    int64  
 1   university      183 non-null    object 
 2   size            115 non-null    float64
 3   bachelors       168 non-null    float64
 4   doctorate       159 non-null    float64
 5   placementscore  183 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 8.7+ KB


In [10]:
dfplacement.sample(3)

,num,university,size,bachelors,doctorate,placementscore
119,120,Georgetown University,20.0,115.0,130.0,0.89
28,29,Pennsylvania State University,65.0,31.0,28.0,1.87
8,9,University of Michigan,97.0,7.0,13.0,3.69


In [11]:
dfbestpaper = pd.read_csv("data/best-paper.csv", delimiter='|')
dfbestpaper = dfbestpaper.applymap(lambda x: x.strip() if isinstance(x, str) else x)
dfbestpaper.describe()

,Year,Coauthors
count,4815.000000,4815.000000
mean,2014.076843,5.455659
std,5.956105,3.199965
min,1996.000000,1.000000
25%,2010.000000,3.000000
50%,2015.000000,5.000000
75%,2019.000000,7.000000
max,2022.000000,26.000000


In [12]:
dfbestpaper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Author      4815 non-null   object
 1   University  4815 non-null   object
 2   Conference  4815 non-null   object
 3   Area        4815 non-null   object
 4   Year        4815 non-null   int64 
 5   Coauthors   4815 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 225.8+ KB


In [13]:
dfbestpaper.sample(3)

,Author,University,Conference,Area,Year,Coauthors
3808,Luca Telloli,Yahoo,CIKM,Knowledge Management,2009,5
2996,Hrvoje Benko,Microsoft,CHI,Human-Computer Interaction,2013,4
2618,Andrew C. Myers,Cornell University,PLDI,Programming Languages,2015,4


In [14]:
dfpreference = pd.read_csv("data/preferences.csv")
dfpreference.describe()

,preference
count,99.000000
mean,2.747475
std,1.311981
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [15]:
dfpreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   conference  99 non-null     object
 1   preference  99 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [16]:
dfpreference.sample(3)

,conference,preference
76,vr,1
24,micro,1
65,iccad,1


In [17]:
university_alias = {}
with open('data/universities.csv', 'r') as file:
    for line in file:
        unis = line.split(',')
        unis.pop()
        first = unis.pop(0)
        university_alias[first] = unis 
count = 5
for k, v in university_alias.items():
    if count < 0:
        break
    count -= 1
    print(f"{k} {v}")
print(len(university_alias))

University of Kentucky []
Colorado State University []
University of Illinois at Chicago []
Duke University []
Florida International University []
UCCS []
186


In [18]:
def get_alias(uni):
    if uni in university_alias:
        return uni
    for k, v in university_alias.items():
        if uni in v:
            return k
    return None 

# Methodology

I want to do a little better than following the rankings blindly, here are the factors I want to consider.

## Data

1. Professor Conference Score (essentially the csrankings.org score)
2. Best Paper Awards
3. Placement Rank (taken from the drafty placement rank, which is pagerank on professors original universities and then employment universities)
4. Variance of Professors

## Method

### Professor Score
A professor will be given a score based on their conference score and best papers. 

Both will have weights by:
- Will be weighted by logarithmically by the year published (the older, the less score)
- Will be weighted by how interested I am in the area (weights defined manually)

### Meta Scores

#### Placement Rank
A Placement Rank will be computed based on the professor score (the places where more important professors originate from matters more).

Logarithmic scale will be used to normalize the professor scores here.

#### Professor Score Variance
Variance of professor score will then be computed

#### Professor Score Sum
A sum of professors score will then be computed

### Final Score

I will take the Meta Scores, normalize them with min max scaling. Then I will apply some weights and get my results.



# Computing Professor Scores

- I will use the `generated-author-info` dataset for each professors to give their scores for each unique year-conference combination
- I will use the `best-paper` dataset for each professor to give a bonus for each professor with a best paper
- I will use `preference` dataset (which I manually created) to give weight preferences.


In [19]:
def year_weight(year):
    # 1970 - 2023
    year = 2023 - year + 3
    year *= 0.5
    return math.log(year)
[year_weight(i) * 0.333 for i in range(1970, 2024)]

[1.1096241018883428,
 1.103623939555951,
 1.0975136763794417,
 1.0912891960863949,
 1.0849461471611537,
 1.0784799245707386,
 1.0718856496811109,
 1.065158148144377,
 1.058291925505866,
 1.0512811402429878,
 1.044119573904407,
 1.0368005979670547,
 1.0293171369683194,
 1.0216616273994945,
 1.0138259717618998,
 1.0058014870860728,
 0.997578847093479,
 0.9891480170347104,
 0.9804981800624246,
 0.971617653784065,
 0.9624937953794228,
 0.9531128933495131,
 0.94346004357072,
 0.9335190068418762,
 0.9232720445058472,
 0.912699727967092,
 0.901780716967036,
 0.8904915002590341,
 0.8788060907618811,
 0.8666956652529798,
 0.8541281360346917,
 0.8410676385546492,
 0.8274739143794042,
 0.8133015627779451,
 0.7984991258418574,
 0.7830079606354381,
 0.7667608359670173,
 0.749680168935963,
 0.7316757842529612,
 0.7126420324442582,
 0.6924540333793854,
 0.6709627058405742,
 0.6479880796354194,
 0.6233101249082299,
 0.5966559032529424,
 0.5676811147153956,
 0.5359428248405554,
 0.5008577731264994,
 0.

In [20]:
def area_weight(area):
    # score of 1-5
    area_score = dfpreference[dfpreference['conference'] == area]['preference'].values[0]
    return math.log(area_score + 1, 8)
[math.log(i+1, math.e ** 0.125)*0.125 for i in range(1,6)]

[0.6931471805599453,
 1.0986122886681098,
 1.3862943611198906,
 1.6094379124341003,
 1.791759469228055]

In [21]:
dfauthorscore = pd.DataFrame(columns=['name', 'university', 'score'])

for index, row in tqdm(dfcsrankings.iterrows(), desc="Prof Scores", unit="item", total=len(dfcsrankings)):
    name = row['name']
    university = row['dept']

    year = row['year']
    area = row['area']

    yearweight = year_weight(year)
    areaweight = area_weight(area)


    count = row['adjustedcount'] * yearweight * areaweight

    existing_rows = dfauthorscore[dfauthorscore['name'] == name]
    if existing_rows.empty:
        dfauthorscore.loc[len(dfauthorscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfauthorscore.at[idx, 'score'] += count

Prof Scores:   0%|                                                                                                               | 0/199906 [00:00<?, ?item/s]

Prof Scores:   0%|                                                                                                     | 4/199906 [00:00<1:26:40, 38.44item/s]

Prof Scores:   0%|                                                                                                     | 36/199906 [00:00<16:37, 200.44item/s]

Prof Scores:   0%|                                                                                                     | 80/199906 [00:00<11:03, 301.34item/s]

Prof Scores:   0%|                                                                                                    | 116/199906 [00:00<10:23, 320.20item/s]

...

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████▉| 199837/199906 [14:15<00:00, 170.06item/s]

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████▉| 199856/199906 [14:15<00:00, 173.89item/s]

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████▉| 199874/199906 [14:15<00:00, 172.06item/s]

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████▉| 199892/199906 [14:15<00:00, 167.38item/s]

Prof Scores: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 199906/199906 [14:15<00:00, 233.66item/s]

In [22]:
dfpaperscore = pd.DataFrame(columns=['name', 'university', 'score'])

for index, row in tqdm(dfbestpaper.iterrows(), desc="Best Paper Scores", unit="item", total=len(dfbestpaper)):
    name = row['Author']
    university = row['University']

    year = row['Year']
    area = row['Area']
    coauthors = 1.0 / float(row['Coauthors'])

    yearweight = year_weight(year)
    areaweight = area_weight(area)

    count = 10 * yearweight * areaweight * coauthors

    existing_rows = dfpaperscore[dfpaperscore['name'] == name]
    if existing_rows.empty:
        dfpaperscore.loc[len(dfpaperscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfpaperscore.at[idx, 'score'] += count

Best Paper Scores:   0%|                                                                                                           | 0/4815 [00:00<?, ?item/s]

Best Paper Scores:   0%|▎                                                                                                | 13/4815 [00:00<00:38, 125.90item/s]

Best Paper Scores:   1%|▌                                                                                                | 30/4815 [00:00<00:31, 151.35item/s]

Best Paper Scores:   1%|▉                                                                                                | 48/4815 [00:00<00:29, 161.55item/s]

Best Paper Scores:   1%|█▎                                                                                               | 67/4815 [00:00<00:27, 169.59item/s]

...

Best Paper Scores:  99%|█████████████████████████████████████████████████████████████████████████████████████████████▉ | 4759/4815 [00:26<00:00, 164.27item/s]

Best Paper Scores:  99%|██████████████████████████████████████████████████████████████████████████████████████████████▏| 4776/4815 [00:26<00:00, 162.19item/s]

Best Paper Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████▌| 4793/4815 [00:26<00:00, 161.75item/s]

Best Paper Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████▉| 4810/4815 [00:26<00:00, 161.46item/s]

Best Paper Scores: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 4815/4815 [00:27<00:00, 178.21item/s]

In [23]:
dfauthorscore.describe()

,score
count,16202.000000
mean,5.289280
std,8.421237
min,0.007387
25%,0.831027
50%,2.286453
75%,6.095181
max,109.848482


In [24]:
dfauthorscore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16202 entries, 0 to 16201
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        16202 non-null  object 
 1   university  16202 non-null  object 
 2   score       16202 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1022.4+ KB


In [25]:
dfauthorscore.sample(3)

,name,university,score
14664,Wensheng Dou,Chinese Academy of Sciences,2.385050
10040,Nathan Jacobs,Washington University in St. Louis,5.654553
10740,Pedro R. D'Argenio,Universidad Nacional de Córdoba,0.669921


In [26]:
dfpaperscore.describe()

,score
count,4099.000000
mean,3.634394
std,4.174043
min,0.169683
25%,1.417166
50%,2.366560
75%,3.981688
max,50.220182


In [27]:
dfpaperscore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4099 entries, 0 to 4098
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        4099 non-null   object 
 1   university  4099 non-null   object 
 2   score       4099 non-null   float64
dtypes: float64(1), object(2)
memory usage: 128.1+ KB


In [28]:
dfpaperscore.sample(3)

,name,university,score
3686,Gaurav Sharma,Purdue University,5.940486
3803,Alexandru Salcianu,Massachusetts Institute of Technology,4.222860
3975,Kalervo Järvelin & Jaana Kekäläinen,University of Tampere,17.099662


In [90]:
dfscore = dfauthorscore.copy()
dfscore.rename(columns={'score': 'authorscore'}, inplace=True)
dfscore['paperscore'] = np.nan

for index, row in tqdm(dfpaperscore.iterrows(), desc="Merge Scores", unit="item", total=len(dfpaperscore)):
    name = row['name']
    university = row['university']
    count = row['score']

    existing_rows = dfscore[dfscore['name'] == name]

    if existing_rows.empty:
        dfscore.loc[len(dfscore)] = [name, university, None, count]
    else:
        for idx in existing_rows.index:
            dfscore.at[idx, 'paperscore'] = count

Merge Scores:   0%|                                                                                                                | 0/4099 [00:00<?, ?item/s]

Merge Scores:   0%|▎                                                                                                     | 11/4099 [00:00<00:38, 104.92item/s]

Merge Scores:   1%|▌                                                                                                     | 22/4099 [00:00<00:37, 107.51item/s]

Merge Scores:   1%|▊                                                                                                     | 33/4099 [00:00<00:39, 102.26item/s]

Merge Scores:   1%|█                                                                                                     | 44/4099 [00:00<00:39, 101.76item/s]

...

Merge Scores:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████▏| 4067/4099 [00:40<00:00, 98.73item/s]

Merge Scores:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████▍| 4077/4099 [00:40<00:00, 94.91item/s]

Merge Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████▋| 4087/4099 [00:40<00:00, 95.04item/s]

Merge Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████▉| 4097/4099 [00:40<00:00, 94.38item/s]

Merge Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 4099/4099 [00:40<00:00, 100.02item/s]

In [91]:
dfscore['university'] = dfscore['university'].apply(get_alias)
dfscore = dfscore.dropna(subset=['university'])
dfscore['name_lower'] = dfscore['name'].str.lower()
dfscore.describe()

,authorscore,paperscore
count,6224.000000,1732.000000
mean,7.105932,4.113517
std,10.088763,4.573070
min,0.007792,0.169683
25%,1.289504,1.527151
50%,3.424615,2.682397
75%,8.729862,4.622596
max,109.848482,50.220182


In [92]:
dfscore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7493 entries, 4 to 19418
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         7493 non-null   object 
 1   university   7493 non-null   object 
 2   authorscore  6224 non-null   float64
 3   paperscore   1732 non-null   float64
 4   name_lower   7493 non-null   object 
dtypes: float64(2), object(3)
memory usage: 351.2+ KB


In [93]:
dfscore.sample(3)

,name,university,authorscore,paperscore,name_lower
15254,YangQuan Chen,Univ. of California - Merced,2.520514,NaN,yangquan chen
950,Anind K. Dey,University of Washington,22.026452,NaN,anind k. dey
9911,Mustafa Bilgic 0001,Illinois Institute of Technology,4.664780,NaN,mustafa bilgic 0001


# Computing Professor Placement Scores

I will compute three scores, one for relationship between bachelor with placement, bachelor with phd, and phd with placement.

I will do this by mapping dfcsprofs onto dfscore, which will let me value each one of the connections.

In [33]:
dfscoreplacement = dfscore.copy()
dfscoreplacement['name'] = dfscoreplacement['name'].str.lower()
dfcsprofsplacement = dfcsprofs.copy()
dfcsprofsplacement['name'] = dfcsprofsplacement['FullName'].str.lower()
dfcsprofsplacement['university'] = dfcsprofsplacement['University'].apply(get_alias)
dfcsprofsplacement['bachelors'] = dfcsprofsplacement['Bachelors'].apply(get_alias)
dfcsprofsplacement['doctorate'] = dfcsprofsplacement['Doctorate'].apply(get_alias)

mapping_dict = dict(zip(dfcsprofsplacement['name'], zip(dfcsprofsplacement['university'],
                                                        dfcsprofsplacement['bachelors'],
                                                        dfcsprofsplacement['doctorate'])))
dfscoreplacement[['placement', 'bachelors', 'doctorate']] = dfscoreplacement['name'].map(mapping_dict).apply(pd.Series)

dfscoreplacement.describe()

,authorscore,paperscore
count,6224.000000,1732.000000
mean,7.105932,4.113517
std,10.088763,4.573070
min,0.007792,0.169683
25%,1.289504,1.527151
50%,3.424615,2.682397
75%,8.729862,4.622596
max,109.848482,50.220182


In [34]:
dfscoreplacement.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7493 entries, 4 to 19418
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         7493 non-null   object 
 1   university   7493 non-null   object 
 2   authorscore  6224 non-null   float64
 3   paperscore   1732 non-null   float64
 4   placement    2823 non-null   object 
 5   bachelors    1195 non-null   object 
 6   doctorate    2370 non-null   object 
dtypes: float64(2), object(5)
memory usage: 468.3+ KB


In [35]:
dfscoreplacement.sample(3)

,name,university,authorscore,paperscore,placement,bachelors,doctorate
18049,dan r. k. ports,University of Washington,NaN,2.638868,NaN,NaN,NaN
2953,daniel szafir,University of North Carolina,3.314018,NaN,University of North Carolina,Boston College,None
1672,bir bhanu,Univ. of California - Riverside,19.905095,NaN,None,None,University of Southern California


# University dataframe

Now I will create a dataframe which has all people who currently work at the university, got their bachelors there, or got their doctorate there.

In [94]:
unique_unis = np.unique(np.concatenate([dfscoreplacement['university'].dropna(),
                                        dfscoreplacement['bachelors'].dropna(),
                                        dfscoreplacement['doctorate'].dropna()]))

length = len(unique_unis)
inner_arrays = []
for _ in range(length):
    inner_arrays.append(np.array([]))

dfuniversity = pd.DataFrame({
    'university': unique_unis,
    'professors': inner_arrays,
    'bachelors': inner_arrays,
    'doctorate': inner_arrays
})
dfuniversity.sample(3)

,university,professors,bachelors,doctorate
112,University of Colorado - Denver,[],[],[]
64,Ohio University,[],[],[]
87,UNC - Charlotte,[],[],[]


In [95]:
for index, row in tqdm(dfscoreplacement.iterrows(), desc="Merge Scores", unit="item", total=len(dfscoreplacement)):
    name = row['name']
    university = row['university']
    bachelors = row['bachelors']
    doctorate = row['doctorate']

    existing_rows = dfuniversity[dfuniversity['university'] == university]
    if not existing_rows.empty:
        for idx in existing_rows.index:
            dfuniversity.at[idx, 'professors'] = np.append(dfuniversity.at[idx, 'professors'], name)

    existing_rows = dfuniversity[dfuniversity['university'] == bachelors]
    if not existing_rows.empty:
        for idx in existing_rows.index:
            dfuniversity.at[idx, 'bachelors'] = np.append(dfuniversity.at[idx, 'bachelors'], name)

    existing_rows = dfuniversity[dfuniversity['university'] == doctorate]
    if not existing_rows.empty:
        for idx in existing_rows.index:
            dfuniversity.at[idx, 'doctorate'] = np.append(dfuniversity.at[idx, 'doctorate'], name)


Merge Scores:   0%|                                                                                                                | 0/7493 [00:00<?, ?item/s]

Merge Scores:   0%|▌                                                                                                     | 37/7493 [00:00<00:20, 362.50item/s]

Merge Scores:   1%|█                                                                                                     | 75/7493 [00:00<00:19, 371.34item/s]

Merge Scores:   2%|█▌                                                                                                   | 113/7493 [00:00<00:21, 340.93item/s]

Merge Scores:   2%|██                                                                                                   | 151/7493 [00:00<00:20, 354.80item/s]

...

Merge Scores:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████▉  | 7337/7493 [00:19<00:00, 408.56item/s]

Merge Scores:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████▍ | 7379/7493 [00:19<00:00, 411.40item/s]

Merge Scores:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████ | 7421/7493 [00:19<00:00, 407.54item/s]

Merge Scores: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████▌| 7462/7493 [00:19<00:00, 403.78item/s]

Merge Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 7493/7493 [00:19<00:00, 381.02item/s]

In [60]:
dfuniversity.sample(3)

,university,professors,bachelors,doctorate
37,Indiana State University,"[arash rafiey, lászló egri]",[],[]
105,University of Arizona,"[alon efrat, beichuan zhang 0001, chicheng zha...","[christian s. collberg, daniel fried, dawson r...","[c. lee giles, david k. lowenthal, jack w. dav..."
15,Clemson University,"[alexander herzog, amy w. apon, andrew c. robb...","[eric patterson, josiah d. hester]","[eric patterson, gregg rothermel, josiah d. he..."


In [62]:
dfuniversity['authorscore'] = inner_arrays
dfuniversity['paperscore'] = inner_arrays
dfuniversity['bachauthorscore'] = inner_arrays
dfuniversity['bachpaperscore'] = inner_arrays
dfuniversity['doctauthorscore'] = inner_arrays
dfuniversity['doctpaperscore'] = inner_arrays
dfuniversity.head()


,university,professors,bachelors,doctorate,authorscore,paperscore,bachauthorscore,bachpaperscore,doctauthorscore,doctpaperscore
0,Arizona State University,"[adam doupé, adil ahmad, andréa w. richa, avir...","[adam doupé, amy voida, anthony gitter, cynthi...","[bernd hamann, dejun yang, hongxin hu, jiayu z...",[],[],[],[],[],[]
1,Auburn University,"[akond ashfaque ur rahman, anh nguyen 0002, an...",[edward w. knightly],[],[],[],[],[],[],[]
2,Boise State University,"[amit jain, bogdan dit, casey kennington, cath...",[],[],[],[],[],[],[],[]
3,Boston College,"[donglai wei 0001, edward sciore, emily tucker...","[andrew crotty, bradley hayes, daniel szafir, ...",[],[],[],[],[],[],[]
4,Boston University,"[adam d. smith, alina ene, ashok cutkosky, ass...","[eric nyberg, gabriel parmer, kihong park, leo...","[albert-lászló barabási, khaled harfoush, paul...",[],[],[],[],[],[]


In [109]:
def lookup_scores(names, type):
    values = []
    for name in names:
        match = dfscore[dfscore['name_lower'] == name]
        if not match.empty:
            values.extend(match[type])
        else:
            values.append(None)
    return [ x for x in values if x is not None]

def lookup_author_scores(names):
    return lookup_scores(names, "authorscore")
def lookup_paper_scores(names):
    return lookup_scores(names, "paperscore")

tqdm.pandas(desc="Processing")
dfuniversity['authorscore'] = dfuniversity['professors'].progress_apply(lookup_author_scores)
dfuniversity['paperscore'] = dfuniversity['professors'].progress_apply(lookup_paper_scores)
dfuniversity['bachauthorscore'] = dfuniversity['bachelors'].progress_apply(lookup_author_scores)
dfuniversity['bachpaperscore'] = dfuniversity['bachelors'].progress_apply(lookup_paper_scores)
dfuniversity['doctauthorscore'] = dfuniversity['doctorate'].progress_apply(lookup_author_scores)
dfuniversity['doctpaperscore'] = dfuniversity['doctorate'].progress_apply(lookup_paper_scores)


Processing:   0%|                                                                                                                     | 0/185 [00:00<?, ?it/s]

Processing:   1%|█▏                                                                                                           | 2/185 [00:00<00:16, 11.38it/s]

Processing:   3%|██▉                                                                                                          | 5/185 [00:00<00:09, 18.66it/s]

Processing:   4%|████▋                                                                                                        | 8/185 [00:00<00:11, 15.72it/s]

Processing:   5%|█████▊                                                                                                      | 10/185 [00:00<00:11, 15.64it/s]

...

Processing:  85%|███████████████████████████████████████████████████████████████████████████████████████████▍               | 158/185 [00:04<00:00, 46.65it/s]

Processing:  89%|███████████████████████████████████████████████████████████████████████████████████████████████▍           | 165/185 [00:05<00:00, 37.31it/s]

Processing:  92%|██████████████████████████████████████████████████████████████████████████████████████████████████▉        | 171/185 [00:05<00:00, 24.87it/s]

Processing:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 179/185 [00:05<00:00, 31.00it/s]

Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 185/185 [00:05<00:00, 31.24it/s]

In [110]:
dfuniversity.sample(3)

,university,professors,bachelors,doctorate,authorscore,paperscore,bachauthorscore,bachpaperscore,doctauthorscore,doctpaperscore
59,Northeastern University,"[aanjhan ranganathan, abhi shelat, alan mislov...",[],"[caiwen ding, christos dimoulas, cristina v. l...","[1.7817052786800995, 14.529522848675839, 9.358...","[nan, nan, nan, nan, nan, 4.554021873471788, n...",[],[],"[1.2408708218309543, 2.312817954798986, 6.0145...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]"
24,Drexel University,"[ali shokoufandeh, christopher w. geib, colin ...","[craig a. schroeder, lisa anthony, nora ayanian]","[aylin caliskan, steven w. zucker, yunan chen]","[4.166008169424921, 7.4087086607872275, 0.5360...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.5142721332475226, 1.2864489496398688, 6.177...","[nan, 4.159560393114647, nan]","[2.628220337309796, 22.3103544989276, 4.924821...","[nan, nan, nan]"
100,University at Albany - SUNY,"[charalampos chelmis, chinwe ekenna, dan e. wi...",[],[],"[0.8128291410046559, 1.8802089366818657, 14.66...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],[],[]
